Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/reinforcement-learning/atari-on-distributed-compute/pong_rllib.png)

# Reinforcement Learning in Azure Machine Learning - Pong problem
Reinforcement Learning in Azure Machine Learning is a managed service for running distributed reinforcement learning training and simulation using the open source Ray framework.
This noteboook demonstrates how to use Ray to solve a more complex problem using a more complex setup including Ray RLLib running on multiple compute nodes and using a GPU.
For this example we will train a Pong playing agent on cluster of two NC6 nodes (6 CPU, 1 GPU).

## Pong problem
[Pong](https://en.wikipedia.org/wiki/Pong) is a two-dimensional sports game that simulates table tennis. The player controls an in-game paddle by moving it vertically across the left or right side of the screen. They can compete against another player controlling a second paddle on the opposing side. Players use the paddles to hit a ball back and forth.

<table style="width:50%">
  <tr>
      <th style="text-align: center;"><img src="./images/pong.gif" alt="Pong image" align="middle" margin-left="auto" margin-right="auto"/></th>
  </tr>
  <tr style="text-align: center;">
      <th>Fig 1. Pong game animation (from <a href="https://towardsdatascience.com/intro-to-reinforcement-learning-pong-92a94aa0f84d">towardsdatascience.com</a>).</th>
  </tr>
</table>

The goal here is to train an agent to win an episode of Pong game against opponent with the score of at least 10 points. An episode in Pong runs until one of the players reaches a score of 21. Episodes are a terminology that is used across all the [OpenAI gym](https://www.gymlibrary.dev/environments/atari/pong/) environments that contains a strictly defined task.

Training a Pong agent is a compute-intensive task and this example demonstrates the use of Reinforcement Learning in Azure Machine Learning service to train an agent faster in a distributed, parallel environment. You'll learn more about using the head and the worker compute targets to train an agent in this notebook below.

## Prerequisite

It is highly recommended that the user should go through the [Reinforcement Learning in Azure Machine Learning - Cartpole Problem on Single Compute](../cartpole-on-single-compute/cartpole_sc.ipynb) to understand the basics of Reinforcement Learning in Azure Machine Learning and Ray RLlib used in this notebook.

### Azure Machine Learning SDK
Display the Azure Machine Learning SDK version.

In [ ]:
%matplotlib inline

# Azure Machine Learning core imports
import azureml.core

# Check core SDK version number
print("Azure Machine Learning SDK version: ", azureml.core.VERSION)

### Get Azure Machine Learning workspace
Get a reference to an existing Azure Machine Learning workspace.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.location, ws.resource_group, sep = ' | ')

### Create Azure Machine Learning experiment
Create an experiment to track the runs in your workspace.

In [ ]:
from azureml.core.experiment import Experiment

# Experiment name
experiment_name = 'rllib-pong-multi-node'
exp = Experiment(workspace=ws, name=experiment_name)

### Create compute target

In this example, we show how to set up a compute target for the Ray nodes.

> Note that if you have an AzureML Data Scientist role, you will not have permission to create compute resources. Talk to your workspace or IT admin to create the compute targets described in this section, if they do not already exist.

In [ ]:
from azureml.core.compute import AmlCompute, ComputeTarget

# Choose a name for the Ray cluster
compute_name = 'compute-gpu'
compute_min_nodes = 0
compute_max_nodes = 2

# This example uses GPU VM.
vm_size = 'Standard_NC6s_v3'

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        if compute_target.provisioning_state == 'Succeeded':
            print('found compute target. just use it', compute_name)
        else: 
            raise Exception(
                'found compute target but it is in state', compute_target.provisioning_state)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(
        vm_size=vm_size,
        min_nodes=compute_min_nodes, 
        max_nodes=compute_max_nodes,
    )

    # Create the cluster
    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)
    
    # Can poll for a minimum number of nodes and for a specific timeout. 
    # If no min node count is provided it will use the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
    # For a more detailed view of current AmlCompute status, use get_status()
    print(compute_target.get_status().serialize())

### Create Azure ML Environment

This step creates and registers an Azure ML Environment that includes all of the dependencies needed to run this example, including CUDA drivers Pytorch, RLLib, and associated tools.  This step can take a significant time (30 min) on the first run.

In [ ]:
ray_environment_name = 'pong-gpu'

In [ ]:
import os
from azureml.core import Environment

ray_environment_dockerfile_path = os.path.join(os.getcwd(), 'docker', 'Dockerfile-gpu')

# Build GPU image
ray_gpu_env = Environment. \
    from_dockerfile(name=ray_environment_name, dockerfile=ray_environment_dockerfile_path). \
    register(workspace=ws)
ray_gpu_build_details = ray_gpu_env.build(workspace=ws)

ray_gpu_build_details.wait_for_completion(show_output=True)

### Create reinforcement learning training run

The code below submits the training run using a `ScriptRunConfig`. By providing the
command to run the training, and a `RunConfig` object configured with your
compute target, number of nodes, and environment image to use.

In [ ]:
from azureml.core import RunConfiguration, ScriptRunConfig

experiment_name = 'rllib-pong-multi-node'

experiment = Experiment(workspace=ws, name=experiment_name)
ray_environment = Environment.get(workspace=ws, name=ray_environment_name)

aml_run_config_ml = RunConfiguration(communicator='OpenMpi')
aml_run_config_ml.target = compute_target
aml_run_config_ml.node_count = 2
aml_run_config_ml.environment = ray_environment

script_name='pong_rllib.py'
config_name='pong-impala-vectorized.yaml'

command=[
    'python', script_name,
    '--config', config_name
]

config = ScriptRunConfig(source_directory='./files',
                    command=command,
                    run_config = aml_run_config_ml
                   )
training_run = experiment.submit(config)

### Training configuration
All training parameters (including the Reinforcement Learning algorithm) are set through a single configuration file.  For this example we'll be using the IMPALA algorithm to train an agent to play Atari Pong.
We set `num_workers` to 11 because we have 11 CPUs available for worker nodes (6 CPUs on each of 2 machines, with 1 CPU consumed as a head node).
We set `episode_reward_mean` (under `stop`) to 10 so that we terminate the run once we achieve a reward score of 10.

Here is the configuration we are using for this example:

```yaml
pong:
    env: ALE/Pong-v5
    run: IMPALA
    config:
        num_workers: 11
        num_gpus: 1
        rollout_fragment_length: 50
        train_batch_size: 1000
        num_sgd_iter: 2
        num_multi_gpu_tower_stacks: 2
        env_config:
            frameskip: 1
            full_action_space: false
            repeat_action_probability: 0.0
        stop:
          episode_reward_mean: 10
          total_time_s: 3600
        model:
          dim: 42

```

### Monitor the run

Azure Machine Learning provides a Jupyter widget to show the status of an experiment run. You could use this widget to monitor the status of the runs. The widget shows the list of two child runs, one for head compute target run and one for worker compute target run. You can click on the link under **Status** to see the details of the child run. It will also show the metrics being logged.

In [ ]:
from azureml.widgets import RunDetails

RunDetails(training_run).show()

### Stop the run

To stop the run, call `training_run.cancel()`.

In [ ]:
# Uncomment line below to cancel the run
# training_run.cancel()

### Wait for completion
Wait for the run to complete before proceeding. If you want to stop the run, you may skip this and move to next section below. 

**Note: The run may take anywhere from 30 minutes to 45 minutes to complete.**

In [ ]:
training_run.wait_for_completion()

### Performance of the agent during training

Let's get the reward metrics for the training run agent and observe how the agent's rewards improved over the training iterations and how the agent learns to win the Pong game. 

Collect the episode reward metrics from the worker run's metrics. 

In [ ]:
# Get the reward metrics from training_run
episode_reward_mean = training_run.get_metrics(name='episode_reward_mean')

Plot the reward metrics. 

In [ ]:
import matplotlib.pyplot as plt

plt.plot(episode_reward_mean['episode_reward_mean'])
plt.xlabel('training_iteration')
plt.ylabel('episode_reward_mean')
plt.show()

We observe that during the training over multiple episodes, the agent learns to win the Pong game against opponent with our target of 10 points in each episode of 21 points.
**Congratulations!! You have trained your Pong agent to win a game.**

## Cleaning up
For your convenience, below you can find code snippets to clean up any resources created as part of this tutorial that you don't wish to retain.

In [ ]:
# To archive the created experiment:
#experiment.archive()

# To delete the compute targets:
#head_compute_target.delete()
#worker_compute_target.delete()